In [6]:
import keras # High level library for implementing neural networks using tensorflow or theano as backend
from keras.datasets import cifar10 # Importing the dataset I'm gonna work with
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [7]:
# Setting the required parameters for training
batch_size = 64
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'Saved_models')
model_name = 'cifar10_trained_model.h5'

In [8]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converting class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
model = Sequential() # Implementing a sequential model using Keras

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu')) # Input Layer with ReLU Activator

model.add(Conv2D(32, (3, 3))) # Hidden layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # A maxpooling layer
model.add(Dropout(0.25)) # A dropout layer to prevent model from overfitting, by maintaining bias-variance trade-off

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax')) # Output layer with a softmax activator

In [10]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/100
782/782 [==============================] - 51s 65ms/step - loss: 1.9274 - acc: 0.2915 - val_loss: 1.6858 - val_acc: 0.3997
Epoch 2/100
782/782 [==============================] - 48s 61ms/step - loss: 1.6816 - acc: 0.3851 - val_loss: 1.5195 - val_acc: 0.4518
Epoch 3/100
782/782 [==============================] - 48s 61ms/step - loss: 1.5613 - acc: 0.4300 - val_loss: 1.5421 - val_acc: 0.4446
Epoch 4/100
782/782 [==============================] - 48s 61ms/step - loss: 1.4895 - acc: 0.4600 - val_loss: 1.3618 - val_acc: 0.5079
Epoch 5/100
782/782 [==============================] - 45s 57ms/step - loss: 1.4283 - acc: 0.4820 - val_loss: 1.2875 - val_acc: 0.5392
Epoch 6/100
782/782 [==============================] - 45s 58ms/step - loss: 1.3802 - acc: 0.5055 - val_loss: 1.2517 - val_acc: 0.5511
Epoch 7/100
782/782 [==============================] - 44s 56ms/step - loss: 1.3368 - acc: 0.5205 - val_loss: 1.2055 - val_acc: 0.5688
Epoch 8/100
782/782 

Epoch 61/100
782/782 [==============================] - 44s 56ms/step - loss: 0.7764 - acc: 0.7364 - val_loss: 0.6844 - val_acc: 0.7668
Epoch 62/100
782/782 [==============================] - 43s 55ms/step - loss: 0.7730 - acc: 0.7368 - val_loss: 0.7050 - val_acc: 0.7612
Epoch 63/100
782/782 [==============================] - 45s 57ms/step - loss: 0.7690 - acc: 0.7367 - val_loss: 0.7589 - val_acc: 0.7387
Epoch 64/100
782/782 [==============================] - 44s 57ms/step - loss: 0.7656 - acc: 0.7379 - val_loss: 0.6792 - val_acc: 0.7687
Epoch 65/100
782/782 [==============================] - 44s 56ms/step - loss: 0.7648 - acc: 0.7374 - val_loss: 0.6786 - val_acc: 0.7709
Epoch 66/100
782/782 [==============================] - 45s 58ms/step - loss: 0.7675 - acc: 0.7360 - val_loss: 0.6768 - val_acc: 0.7696
Epoch 67/100
782/782 [==============================] - 45s 57ms/step - loss: 0.7625 - acc: 0.7421 - val_loss: 0.6924 - val_acc: 0.7637
Epoch 68/100
782/782 [==========================

In [11]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\Priyanshu\Documents\Projects\cifar-10-batches-py\Saved_models\cifar10_trained_model.h5 
10000/10000 [==============================] - 1s 99us/step
Test loss: 0.6207268326282501
Test accuracy: 0.7874
